In [1]:
import os
import csv
import pandas as pd

In [10]:
pwd = os.getcwd()
file_paths = ['/scraped_data/scrape_of_rows_odd.csv', '/scrape_data/scraped_rows_even.csv']

In [8]:
def read_to_list(index):
    with open(pwd + file_paths[index], 'r') as f:
        reader = csv.reader(f)
        return list(reader)

In [13]:
odd_rows_raw = read_to_list(0)
even_rows_raw = read_to_list(1)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\worri\\OneDrive\\data_analytics\\max_verstappen\\max_verstappen_stats/scrape_data/scraped_rows_even.csv'

# <p align=center> Pre Processing Data </p>
## <p>Noticing that the data from the website is displayed rather annoyingly, in the form of span rows, I have decided to alternate the rows with the view of pasting together both list into a DataFrame. To acheieve this, I have to pre process the data to have it in the correct sizing, format to import into the DataFrame.  The odd rows in particular, the way the data was displayed resulted in some entries being not of the same length, thus being out of sync with most of the data pulled from this subsection of rows.  The best way I came about to solving this issue, was to isolate the rows that are not needed, in particular the empty lists, and those that took the header of the table, and remove them.  Then insert a generic entry into the lists that remain, to align the data with the rest.  I will go through and clean this data once It is active in the DataFrame</p>

In [15]:
odd_df = pd.DataFrame(odd_rows_raw, columns=["id", "track", "team", "driver_num", "driver", "model", "engine", "tyre", "grid_pos", "fin_pos", "notes"]) 
print(odd_df.shape)

odd_df = odd_df.iloc[odd_df.index[46:68]].shift(periods=2, axis="columns")

(344, 11)


,id,track,team,driver_num,driver,model,engine,tyre,grid_pos,fin_pos,notes
0,n,Grand Prix,Team,N°,Driver,Model,Engine,Tyre,Grid,Race,None
1,None,None,None,None,None,None,None,None,None,None,None
2,167,Australia,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,11,ab,Engine
3,None,None,None,None,None,None,None,None,None,None,None
4,,,,,,,,,,,


In [17]:
#Sanity check for the amount of lists with empty strings. 
len(odd_rows_raw)
count = 0
for ls in odd_rows_raw:
    if ls[0] == '':
        count += 1
print(count)

IndexError: list index out of range